In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import ccxt
from sqlalchemy import create_engine
import os
from datetime import datetime, timedelta


with open("../api(JKBOT).txt") as f:
    lines = f.readlines()
    api_key = lines[0].strip()
    secret = lines[1].strip()

binance = ccxt.binance(config={
    'apiKey': api_key, #"XTpKrQGSk3GhXzqiEV4OfwGJzmTVcLh8dKGwHo4aQBH4p0mOqPDpIsxdh95tjGVf",
    'secret': secret, #"A0eqZGEWHsyL3NMM6WuDrucIanr7A2YZAnrwXVPhXpf2WGauIANwa5zsoNeNt0hs",
    'enableRateLimit' : True,
    'options': {
        'defaultType': 'future'
    }
})

jongmok= []

markets = binance.load_markets()
for market in markets.keys():
    if market.endswith("/USDT"):# and market != "BCC/USDT" and market != "TUSD/USDT":
        jongmok.append(market)

database = 'price_1d'

def md_connect(user, password, db, host, port=3306):
    url = 'mysql+pymysql://{}:{}@{}:{}/{}'.format(user, password, host, port, db)
    engine = create_engine(url).connect()
    return engine

engine = md_connect('root', '1234', database, 'localhost')

print("database :" + database + " 에 접속중 입니다")

score = []
window_list = []
away_list = []
size=30
term = '1d'
window = 28
away = 7
# # for away in range(1, 14):
# print("window : " + str(window), "away : " + str(away))


database :price_1d 에 접속중 입니다


In [ ]:
#모델 만들기

x_input = []
y_target = []
for symbol in jongmok[0:80]:
    data = pd.read_sql(symbol[:-5].lower(), engine, index_col='datetime') #sql에서 db 받아오기
    x_data = data[['open', 'high', 'low', 'close']]
    y_data = data[['close']]
    print(symbol + " Data Collect")
    for i in range(0, len(data) - window - away + 1): #len(data)-window
        mean = x_data[i:i+window].mean()
        std = x_data[i:i+window].std()
        x = (x_data[i:i+window] - mean) / std
        # x['volume'] = (x_data['volume'] - volume_mean) / (volume_std)
        x_input.append(x)

        if y_data['close'][i+window-1] < y_data['close'][i+window+away-1]:
            y_target.append(1)
        else:
            y_target.append(0)

x_input = np.array(x_input)
y_target = np.array(y_target)

train_input, test_input, train_target, test_target = train_test_split(x_input, y_target, test_size=0.5)
from tensorflow import keras
model = keras.Sequential()
model.add(keras.layers.LSTM(size, activation='relu', input_shape=(window, 4), dropout=0.3))
# model.add(keras.layers.LSTM(10, activation = 'relu', input_shape=(window, 4)))
# model.add(keras.layers.Dropout(0, 1))
# model.add(keras.layers.LSTM(8))
model.add(keras.layers.Dense(1, activation='sigmoid'))

# rmsprop = keras.optimizers.RMSprop(learning_rate=1e-4)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics='accuracy')
checkpoint_cb = keras.callbacks.ModelCheckpoint('h5/'+ term + '_' +str(window) + '_' + str(away) + '_(' + str(size) + ').h5', monitor='val_loss', save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)
epochs = 100
print("window : " + str(window), "away : " + str(away), "에 대해 "+ str(epochs) +"회 학습중")

model.fit(train_input, train_target, validation_data=(test_input, test_target), epochs=epochs, verbose=1, callbacks=[checkpoint_cb, early_stopping_cb])
print(window, away, model.evaluate(test_input, test_target)[1])

away_list.append([window, away, model.evaluate(test_input, test_target)[1]])
tot_score = pd.DataFrame(away_list)
with pd.ExcelWriter('excel/' + term + '_' +str(window) + '_' + str(away) + '_(' + str(size) + ').xlsx') as writer:
    tot_score.to_excel(writer, index=False)



In [3]:
from sqlalchemy import create_engine
# sql 연결 함수
def md_connect(user, password, db, host, port=3306):
    url = 'mysql+pymysql://{}:{}@{}:{}/{}'.format(user, password, host, port, db)
    engine = create_engine(url).connect()
    return engine

# 백테스팅 결과 함수
def select_rate(long_std, short_std, start_date=0, end_date=0, fee=0):
    df = ind_data[(ind_data['ind'] > long_std) | (ind_data['ind'] < short_std)].sort_index()
    condition = df['ind'] > 0.5
    df = df.assign(position=condition.map({True:1, False:-1}))
    df['profit'] = df['position'] * df['rate'] -(2 * fee)

    if start_date != 0:
        if end_date != 0:
            df = df.loc[start_date:end_date]
        else:
            df = df.loc[start_date:]
    else:
        if end_date !=0:
            df = df.loc[:end_date]

    df['count_sum'] = df.groupby('datetime').count()['sym'].rolling(window=7, min_periods=1).sum()
    df['tot_profit'] = df['profit'].cumsum()

    return df

'''매매기록일지'''
log_column = ['시작', '끝', '거래기간', '매수조건', '매도조건', '거래 횟수', '승률', '승수', '패수', '평균보유량', '평균수익률', '총수익률']
log_df = pd.DataFrame(columns = log_column)
engine = md_connect('root', '1234', 'price_1d', 'localhost')
''''''
#가격데이터 연결

account = [] # 계좌 정보
diary = [] # 매매일지
tot_data = {}

# 가격데이터 가져오기
for sym in jongmok[:80]:
    sym = sym[:-5].lower()
    data = pd.read_sql(sym, engine, index_col='datetime') # 데이터 가져오기
    tot_data[sym] = data



In [4]:
# predict 모델 가져오기
from tensorflow import keras
model = keras.Sequential()
model.add(keras.layers.LSTM(30, activation='relu', input_shape=(window, 4), dropout=0.3))
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.load_weights('h5/'+ term + '_' +str(window) + '_' + str(away) + '_(' + str(size) + ').h5')

engine = md_connect('root', '1234', 'ind_test', 'localhost')


In [5]:
# 예측하기
for key in jongmok[:80]:
    key = key[:-5].lower()
    data = tot_data[key]
    x_data = tot_data[key][['open', 'high', 'low', 'close']]
    y_data = tot_data[key][['close']]

    x_input = []
    y_input = []

    ## 전처리
    for i in range(len(data)-window-away+1): #len(data)-window
        mean = x_data[i:i+window].mean()
        std = x_data[i:i+window].std()
        x = (x_data[i:i+window] - mean) / std
        x_input.append(x)

    ## ind 지표 생성
    x_input = np.array(x_input)
    predict_data = model.predict(x_input).tolist()

    ## 수익률 생성
    data['rate'] = data['close'].pct_change(periods=away).shift(-away)
    predict = []
    for i in range(len(predict_data)):
        predict.append(predict_data[i][0])
    data = data[window-1:-away]

    ##ind 지표 결합
    data = data.copy()
    data['ind'] = predict
    data['sym'] = key

    engine = md_connect('root', '1234', 'ind_test', 'localhost')
    data = data[['sym', 'open', 'high', 'low', 'close', 'rate', 'ind']]
    data.to_sql(key, engine, if_exists="replace", index=True)             

24/24 [==============================] - 0s 3ms/step


In [7]:
ind_data

NameError: name 'ind_data' is not defined

In [6]:
tot_data

{'btc':                 open      high       low     close      volume      rate
 datetime                                                                
 2019-09-08  10000.00  10412.65  10000.00  10391.63    3096.291 -0.008604
 2019-09-09  10316.62  10475.54  10077.22  10307.00   14824.373 -0.005601
 2019-09-10  10307.00  10382.97   9940.87  10102.02    9068.955  0.008365
 2019-09-11  10094.27  10293.11   9884.31  10159.55   10897.922 -0.000432
 2019-09-12  10163.06  10450.13  10042.12  10415.13   15609.634 -0.016306
 ...              ...       ...       ...       ...         ...       ...
 2023-03-23  27233.80  28777.00  27101.00  28284.10  808174.047       NaN
 2023-03-24  28284.00  28369.10  26960.00  27438.90  696075.370       NaN
 2023-03-25  27439.00  27788.00  27131.80  27447.30  335674.055       NaN
 2023-03-26  27447.20  28186.90  27400.00  27954.30  399756.985       NaN
 2023-03-27  27954.30  28019.40  27650.00  27781.20   83360.794       NaN
 
 [1297 rows x 6 columns],
 'e

In [ ]:
# #today <-
# before = 1
# ind_list = []
# for key in jongmok[:1]:
#     key = key[:-5].lower()
#     data = tot_data[key]
#     x_data = tot_data[key][['open', 'high', 'low', 'close']]
#     y_data = tot_data[key][['close']]

#     x_input = []
#     y_input = []

#     ## 전처리
#     mean = x_data[-28-before:-before].mean()
#     std = x_data[-28-before:-before].std()
#     x = (x_data[-28-before:-before] - mean) / std
#     x_input.append(x)

#     ## ind 지표 생성
#     x_input = np.array(x_input)
#     predict_data = model.predict(x_input).tolist()
#     ind_list.append([key, predict_data[0][0]])


In [5]:
engine = md_connect('root', '1234', 'ind_test', 'localhost')

# ind dataframe에서 가져오기
for sym in jongmok[:80]:
    sym = sym[:-5].lower()
    data = pd.read_sql(sym, engine, index_col='datetime') # 데이터 가져오기
    tot_data[sym] = data

    # 변수 설정
ind_data = []
for key in jongmok[:80]:
    key = key[:-5].lower()
    ind_data.append(tot_data[key][['sym', 'close', 'rate', 'ind']])

ind_data = pd.concat(ind_data)

In [7]:
data = pd.read_sql('btc', engine, index_col='datetime')

In [8]:
data

,sym,open,high,low,close,rate,ind
datetime,,,,,,,


In [11]:
tot_data['btc']

,open,high,low,close,volume
datetime,,,,,
2019-09-08,10000.00,10412.65,10000.00,10391.63,3096.291
2019-09-09,10316.62,10475.54,10077.22,10307.00,14824.373
2019-09-10,10307.00,10382.97,9940.87,10102.02,9068.955
2019-09-11,10094.27,10293.11,9884.31,10159.55,10897.922
2019-09-12,10163.06,10450.13,10042.12,10415.13,15609.634
...,...,...,...,...,...
2023-03-23,27233.80,28777.00,27101.00,28284.10,808174.047
2023-03-24,28284.00,28369.10,26960.00,27438.90,696075.370
2023-03-25,27439.00,27788.00,27131.80,27447.30,335674.055


In [29]:
tot_data['btc']

,sym,open,high,low,close,rate,ind
datetime,,,,,,,


In [22]:
tot_data

{'btc': Empty DataFrame
 Columns: [sym, open, high, low, close, rate, ind]
 Index: [],
 'eth': Empty DataFrame
 Columns: [sym, open, high, low, close, rate, ind]
 Index: [],
 'bch': Empty DataFrame
 Columns: [sym, open, high, low, close, rate, ind]
 Index: [],
 'xrp': Empty DataFrame
 Columns: [sym, open, high, low, close, rate, ind]
 Index: [],
 'eos': Empty DataFrame
 Columns: [sym, open, high, low, close, rate, ind]
 Index: [],
 'ltc': Empty DataFrame
 Columns: [sym, open, high, low, close, rate, ind]
 Index: [],
 'trx': Empty DataFrame
 Columns: [sym, open, high, low, close, rate, ind]
 Index: [],
 'etc': Empty DataFrame
 Columns: [sym, open, high, low, close, rate, ind]
 Index: [],
 'link': Empty DataFrame
 Columns: [sym, open, high, low, close, rate, ind]
 Index: [],
 'xlm': Empty DataFrame
 Columns: [sym, open, high, low, close, rate, ind]
 Index: [],
 'ada': Empty DataFrame
 Columns: [sym, open, high, low, close, rate, ind]
 Index: [],
 'xmr': Empty DataFrame
 Columns: [sym, op

In [7]:
ind_data

,sym,close,rate,ind
datetime,,,,
2021-02-21,sand,0.32220,-0.318281,0.436657
2021-02-22,sand,0.29508,-0.187475,0.466722
2021-02-23,sand,0.24034,0.045768,0.503749
2021-02-24,sand,0.24781,0.241677,0.504568
2021-02-25,sand,0.23265,0.543520,0.486899
...,...,...,...,...
2023-03-16,sand,0.59740,0.105792,0.345176
2023-03-17,sand,0.67850,-0.071481,0.416465
2023-03-18,sand,0.66810,-0.068702,0.484115


In [8]:
import matplotlib.pyplot as plt
start_date = '2022-03-01'
start_date = datetime.strptime(start_date, '%Y-%m-%d')
# end_date = start_date + timedelta(days=30)
end_date = '2023-03-26'
end_date = datetime.strptime(end_date, '%Y-%m-%d')
# end_date = start_date + timedelta(days=30)
# for i in range(22):
long_std = 0.8
short_std = 0.2

df = select_rate(long_std, short_std, start_date, end_date, 0.0004)

win_cnt = (df['profit'] > 0).sum()
lose_cnt = (df['profit'] < 0).sum()

day = end_date - start_date
day.days

# log.append([len(df), day.days, win_cnt / len(df), win_cnt, lose_cnt, df['count_sum'].mean(), df['profit'].mean(), df['profit'].sum()])
log = {'시작' : start_date, '끝' : end_date, 'window' : window, 'away' : away, '매수조건' : long_std, '매도조건' : short_std, '거래 횟수' : len(df), '거래기간' : day.days, '승률' : win_cnt / len(df), '승수' : win_cnt, '패수' : lose_cnt, '평균보유량' : df['count_sum'].mean(), '평균수익률' : df['profit'].mean(), '총수익률' : df['profit'].sum()}
log_df = log_df.append(log, ignore_index=True)

# start_date = end_date + timedelta(days=1)
# end_date = end_date + timedelta(days=31)
with pd.ExcelWriter('log/log_df'+str(away) +str(window) + '.xlsx') as writer:
    log_df.to_excel(writer, index=False)

C:\Users\tomkj\AppData\Local\Temp\ipykernel_27168\2863674081.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  log_df = log_df.append(log, ignore_index=True)


In [9]:
log_df

,시작,끝,거래기간,매수조건,매도조건,거래 횟수,승률,승수,패수,평균보유량,평균수익률,총수익률,window,away
0,2022-03-01,2023-03-26,390,0.8,0.2,43,0.860465,37,6,6.511628,0.129052,5.549256,28.0,7.0


In [ ]:
df = ind_data[(ind_data['ind'] > long_std) | (ind_data['ind'] < short_std)].sort_index()
df

In [ ]:
import keras
import pandas as pd
import numpy as np

from sqlalchemy import create_engine
import matplotlib
from datetime import datetime, timedelta

def md_connect(user, password, db, host, port=3306):
    url = 'mysql+pymysql://{}:{}@{}:{}/{}'.format(user, password, host, port, db)
    engine = create_engine(url).connect()
    return engine

engine = md_connect('root', '1234', 'price_1d', 'localhost')



In [ ]:
account = [] # 계좌 정보
diary = [] # 매매일지

window = 14
away = 7

start_date = ''
end_date = ''

tot_data = {}
engine = md_connect('root', '1234', 'price_1d', 'localhost')
for sym in jongmok[:100]:
    data = pd.read_sql(sym[:-5].lower(), engine, index_col='datetime') # 데이터 가져오기
    tot_data[sym] = data

In [ ]:
from tensorflow import keras
model = keras.Sequential()
model.add(keras.layers.LSTM(50, activation='relu', input_shape=(window, 4), dropout=0.3))
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.load_weights('h5/'+ term + '_' +str(window) + '_' + str(away) + '_(' + str(size) + ').h5')

engine = md_connect('root', '1234', 'ind_test', 'localhost')

for key in jongmok[:100]:
    data = tot_data[key]
    x_data = tot_data[key][['open', 'high', 'low', 'close']]
    y_data = tot_data[key][['close']]

    x_input = []
    y_input = []

    ## 전처리
    for i in range(len(data)-window-away+1): #len(data)-window
        mean = x_data[i:i+window].mean()
        std = x_data[i:i+window].std()
        x = (x_data[i:i+window] - mean) / std
        x_input.append(x)
    #
    ## ind 지표 생성
    x_input = np.array(x_input)
    predict_data = model.predict(x_input).tolist()

    ## 수익률 생성
    data['rate'] = data['close'].pct_change(periods=away).shift(-away)


    predict = []
    for i in range(len(predict_data)):
        predict.append(predict_data[i][0])
    data = data[window-1:-away]

    ##ind 지표 결합
    data['ind'] = predict
    print(len(data))

    data['sym'] = key


    data = data[['sym', 'open', 'high', 'low', 'close', 'rate', 'ind']]


    data.to_sql(key, engine, if_exists="replace", index=True)

In [ ]:
engine = md_connect('root', '1234', 'ind_test', 'localhost')
for sym in jongmok[:80]:
    data = pd.read_sql(sym[:-5].lower(), engine, index_col='datetime') # 데이터 가져오기
    tot_data[sym] = data

In [17]:
# 변수 설정
ind_data = []
for key in jongmok[:80]:
    ind_data.append(tot_data[key][['sym', 'close', 'rate', 'ind']])

ind_data = pd.concat(ind_data)


def select_rate(long_std, short_std, start_date=0, end_date=0, fee=0):
    df = ind_data[(ind_data['ind'] > long_std) | (ind_data['ind'] < short_std)].sort_index()
    condition = df['ind'] > 0.5
    df = df.assign(position=condition.map({True:1, False:-1}))
    df['profit'] = df['position'] * df['rate'] -(2 * fee)

    if start_date != 0:
        if end_date != 0:
            df = df.loc[start_date:end_date]
        else:
            df = df.loc[start_date:]
    else:
        if end_date !=0:
            df = df.loc[:end_date]


    df['count_sum'] = df.groupby('datetime').count()['sym'].rolling(window=7, min_periods=1).sum()
    df['tot_profit'] = df['profit'].cumsum()

    return df

KeyError: 'BTC/USDT'

In [15]:
ind_data

,sym,close,rate,ind
datetime,,,,
2021-02-21,sand,0.32220,-0.318281,0.436657
2021-02-22,sand,0.29508,-0.187475,0.466722
2021-02-23,sand,0.24034,0.045768,0.503749
2021-02-24,sand,0.24781,0.241677,0.504568
2021-02-25,sand,0.23265,0.543520,0.486899
...,...,...,...,...
2023-03-16,sand,0.59740,0.105792,0.345176
2023-03-17,sand,0.67850,-0.071481,0.416465
2023-03-18,sand,0.66810,-0.068702,0.484115


In [ ]:
log_column = ['시작', '끝', '거래기간', 'window', 'away', '매수조건', '매도조건', '거래 횟수', '승률', '승수', '패수', '평균보유량', '평균수익률', '총수익률']
log_df = pd.DataFrame(columns = log_column)

In [13]:
import matplotlib.pyplot as plt
start_date = '2022-01-01'
start_date = datetime.strptime(start_date, '%Y-%m-%d')
end_date = start_date + timedelta(days=30)

for i in range(22):
    long_std = 0.8
    short_std = 0.2

    df = select_rate(long_std, short_std, start_date, end_date, 0.0004)

    win_cnt = (df['profit'] > 0).sum()
    lose_cnt = (df['profit'] < 0).sum()

    day = end_date - start_date
    day.days

    # log.append([len(df), day.days, win_cnt / len(df), win_cnt, lose_cnt, df['count_sum'].mean(), df['profit'].mean(), df['profit'].sum()])
    log = {'시작' : start_date, '끝' : end_date, 'window' : window, 'away' : away, '매수조건' : long_std, '매도조건' : short_std, '거래 횟수' : len(df), '거래기간' : day.days, '승률' : win_cnt / len(df), '승수' : win_cnt, '패수' : lose_cnt, '평균보유량' : df['count_sum'].mean(), '평균수익률' : df['profit'].mean(), '총수익률' : df['profit'].sum()}
    log_df = log_df.append(log, ignore_index=True)

    start_date = end_date + timedelta(days=1)
    end_date = end_date + timedelta(days=31)



C:\Users\tomkj\AppData\Local\Temp\ipykernel_27168\252574245.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  log_df = log_df.append(log, ignore_index=True)
C:\Users\tomkj\AppData\Local\Temp\ipykernel_27168\252574245.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  log_df = log_df.append(log, ignore_index=True)
C:\Users\tomkj\AppData\Local\Temp\ipykernel_27168\252574245.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  log_df = log_df.append(log, ignore_index=True)
C:\Users\tomkj\AppData\Local\Temp\ipykernel_27168\252574245.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  log_df = log_df.append(log, ignore_index=True)
C:\U

In [14]:
log_df

,시작,끝,거래기간,매수조건,매도조건,거래 횟수,승률,승수,패수,평균보유량,평균수익률,총수익률,window,away
0,2022-03-01,2023-03-26,390,0.8,0.2,43,0.860465,37,6,6.511628,0.129052,5.549256,28.0,7.0
1,2023-01-01,2023-01-31,30,0.8,0.2,3,1.000000,3,0,2.000000,0.299822,0.899467,28.0,7.0
2,2023-02-01,2023-03-03,30,0.8,0.2,0,NaN,0,0,NaN,NaN,0.000000,28.0,7.0
3,2023-03-04,2023-04-03,30,0.8,0.2,0,NaN,0,0,NaN,NaN,0.000000,28.0,7.0
4,2023-04-04,2023-05-04,30,0.8,0.2,0,NaN,0,0,NaN,NaN,0.000000,28.0,7.0
5,2023-05-05,2023-06-04,30,0.8,0.2,0,NaN,0,0,NaN,NaN,0.000000,28.0,7.0
6,2023-06-05,2023-07-05,30,0.8,0.2,0,NaN,0,0,NaN,NaN,0.000000,28.0,7.0
7,2023-07-06,2023-08-05,30,0.8,0.2,0,NaN,0,0,NaN,NaN,0.000000,28.0,7.0
8,2023-08-06,2023-09-05,30,0.8,0.2,0,NaN,0,0,NaN,NaN,0.000000,28.0,7.0
9,2023-09-06,2023-10-06,30,0.8,0.2,0,NaN,0,0,NaN,NaN,0.000000,28.0,7.0


In [ ]:
start_date = '2022-01-01'

In [ ]:
from datetime import datetime, timedelta

In [ ]:
start_date = datetime.strptime(start_date, '%Y-%m-%d')

In [ ]:
start_date = start_date + timedelta(days=90)

In [ ]:
start_date